<a href="https://colab.research.google.com/github/finardi/tutos/blob/master/%5BFinal%5D_llamaindex_retrieval_chunk_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img alt="arize llama-index logos" src="https://storage.googleapis.com/arize-assets/phoenix/assets/docs/notebooks/llama-index-knowledge-base-tutorial/arize_llamaindex.png" width="400">



## LlamaIndex Chunk Size, Retrieval Method and K Eval Suite

This colab provides a suite of retrieval performance tests that helps teams understand
how to setup the retrieval system. It makes use of the Phoenix Eval options for
Q&A (overall did it answer the question) and retrieval (did the right chunks get returned).

There is a sweep of parameters that is stored in experiment_data/results_no_zero_remove,
check that directory for results.

The goal is to help teams choose a Chunk size, retireval method, K for return chunks

This colab downloads the script (py) files. Those files can be run without this colab directly,
in a code only environment (VS code for example)

### Retrieval Eval

This Eval evaluates whether a retrieved chunk contains an answer to the query. Its extremely useful for evaluating retrieval systems.

https://docs.arize.com/phoenix/concepts/llm-evals/retrieval-rag-relevance


### Q&A EVal
This Eval evaluates whether a question was correctly answered by the system based on the retrieved data. In contrast to retrieval Evals that are checks on chunks of data returned, this check is a system level check of a correct Q&A.

https://docs.arize.com/phoenix/concepts/llm-evals/q-and-a-on-retrieved-data

<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-assets/phoenix/assets/images/chunking.png" width=800/>
    </p>
</center>

The challenge in setting up a retrieval system is having solid performance metrics that allow you to evaluate your different strategies:
- Chunk Size
- Retrieval Method
- K value

In setting the above variables you first need some overall Eval metrics.

<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-assets/phoenix/assets/images/eval_relevance.png" width=800/>
    </p>
</center>

The above is the relevance Evaluation used to check on whether the chunk retrieved is relevant to the query.

<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-assets/phoenix/assets/images/EvalQ_A.png" width=600/>
    </p>
</center>

The above Eval shows a Q&A Eval on the entire system Q&A /
on the overall question and answer.
Each is used as we sweep through parameters to detremine effectiveness of retrieval.

## Sweeping values
The scripts sweep through K, Retrival approach and chunk size, determining the trade off on your own docs.

<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-assets/phoenix/assets/images/sweep_k.png" width=800/>
    </p>
</center>

The above shows sweeping through K=4 and K=6

<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-assets/phoenix/assets/images/sweep_chunk.png" width=800/>
    </p>
</center>

The above shows sweeping through Chunk Size

In [ ]:
#The script below runs a test on the question set, by default we have a 170 Question set
#That takes some time to run so you can default it lower just to test
#Comment this out to run on full dataset
QUESTION_SAMPLES = 5

In [ ]:
##############################
### PARAMETER SWEEPS BELOW ###
##############################
###chunk_sizes### to test, will sweep through values of chunk size
chunk_sizes = [
    100,
    # 300,
    # 500,
    # 1000,
    # 2000,
]  # change this, perhaps experiment from 500 to 3000 in increments of 500

### K ###: Sizes to test, will sweep through values of k
k = [4, 6, 10]
# k = [10]  # num documents to retrieve

### Retrieval Approach ###: transformation to test will sweep through retrieval
# transformations = ["original", "original_rerank","hyde", "hyde_rerank"]
transformations = ["original", "original_rerank"]
#Model for Q&A
llama_index_model = "gpt-4"
# llama_index_model = "gpt-3.5-turbo"

In [ ]:
!pip install -q cohere matplotlib lxml openai arize-phoenix llama_index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 850.8/850.8 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.7/2.7 MB 21.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 33.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.0/269.0 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.8/90.8 kB 9.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

# Retrieval Eval Scripts
The following scripts can be run directly. In the case of long test suites, we recommend running
the python script llama_index_w_evals_and_qa directly.py directly in python. All parameters are available
in that script.

In [ ]:
# # Donwload scripts
import requests

url = "https://raw.githubusercontent.com/Arize-ai/phoenix/main/scripts/rag/llama_index_w_evals_and_qa.py"
response = requests.get(url)
with open("llama_index_w_evals_and_qa.py", "w") as file:
     file.write(response.text)

url = "https://raw.githubusercontent.com/Arize-ai/phoenix/main/scripts/rag/plotresults.py"
response = requests.get(url)
with open("plotresults.py", "w") as file:
     file.write(response.text)

In [ ]:
import datetime
import os
import pickle
import pandas as pd

import cohere
import openai
import phoenix.experimental.evals.templates.default_templates as templates
from phoenix.experimental.evals import OpenAIModel
from llama_index import download_loader
from llama_index_w_evals_and_qa import get_urls, plot_graphs, run_experiments

In [ ]:
# @title Default title text
import os
os.environ['OPENAI_API_KEY'] = ""
openai.api_key = ""
os.environ["COHERE_API_KEY"] = ""
cohere.api_key = os.getenv("COHERE_API_KEY")

# Phoenix Observabiility
Click link below to visualize llamaIndex queries and chunking as its happening!!!!!

In [ ]:
#########################################
### CLICK LINK BELOW FOR PHOENIX VIZ ####
#########################################

# Phoenix can display in real time the traces automatically
# collected from your LlamaIndex application.
import phoenix as px
# Look for a URL in the output to open the App in a browser.
px.launch_app()
# The App is initially empty, but as you proceed with the steps below,
# traces will appear automatically as your LlamaIndex application runs.

import llama_index
llama_index.set_global_handler("arize_phoenix")

🌍 To view the Phoenix app in your browser, visit https://5stixt52j21-496ff2e9c6d22116-6060-colab.googleusercontent.com/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


In [ ]:
openai.api_key = os.getenv("OPENAI_API_KEY")
cohere.api_key = os.getenv("COHERE_API_KEY")

# if loading from scratch, change these below
web_title = "arize"  # nickname for this website, used for saving purposes
base_url = "https://docs.arize.com/arize"
# Local files
file_name = "raw_documents.pkl"
save_base = "./experiment_data/"
if not os.path.exists(save_base):
    os.makedirs(save_base)

run_name = datetime.datetime.now().strftime("%Y%m%d_%H%M")
save_dir = os.path.join(save_base, run_name)
if not os.path.exists(save_dir):
    # Create a new directory because it does not exist
    os.makedirs(save_dir)


questions = pd.read_csv(
    "https://storage.googleapis.com/arize-assets/fixtures/Embeddings/GENERATIVE/constants.csv",
    header=None,
)[0].to_list()

In [ ]:
#This will determine run time, how many questions to pull from the data to run
selected_questions = questions[:QUESTION_SAMPLES] if QUESTION_SAMPLES else questions

In [ ]:
#!pip install --force-reinstall urllib3
!pip uninstall urllib3 -y
!pip install urllib3==2.0.4


Found existing installation: urllib3 1.26.16
Uninstalling urllib3-1.26.16:
  Successfully uninstalled urllib3-1.26.16
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.9/123.9 kB 2.5 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index 0.8.37 requires urllib3<2, but you have urllib3 2.0.4 which is incompatible.


In [ ]:
raw_docs_filepath = os.path.join(save_base, file_name)
if not os.path.exists(raw_docs_filepath):
    print(f"'{raw_docs_filepath}' does not exists.")
    urls = get_urls(base_url)  # you need to - pip install lxml
    print(f"LOADED {len(urls)} URLS")

print("GRABBING DOCUMENTS")
BeautifulSoupWebReader = download_loader("BeautifulSoupWebReader")
# two options here, either get the documents from scratch or load one from disk
if not os.path.exists(raw_docs_filepath):
    print("LOADING DOCUMENTS FROM URLS")
    # You need to 'pip install lxml'
    loader = BeautifulSoupWebReader()
    documents = loader.load_data(urls=urls)  # may take some time
    with open(save_base + file_name, "wb") as file:
        pickle.dump(documents, file)
    print("Documents saved to raw_documents.pkl")
else:
    print("LOADING DOCUMENTS FROM FILE")
    print("Opening raw_documents.pkl")
    with open(save_base + file_name, "rb") as file:
        documents = pickle.load(file)

#Model for Evals
eval_model = OpenAIModel(model_name="gpt-4", temperature=0.0)

qa_template = templates.QA_PROMPT_TEMPLATE_STR

'./experiment_data/raw_documents.pkl' does not exists.
LOADED 171 URLS
GRABBING DOCUMENTS
LOADING DOCUMENTS FROM URLS


Documents saved to raw_documents.pkl


In [ ]:
# Uncomment when testing, 3 questions are easy to run through quickly
# questions = questions[0:3]
all_data = run_experiments(
    documents=documents,
    queries=selected_questions,
    chunk_sizes=chunk_sizes,
    query_transformations=transformations,
    k_values=k,
    web_title=web_title,
    save_dir=save_dir,
    llama_index_model=llama_index_model,
    eval_model=eval_model,
    template=qa_template,
)

[nltk_data] Downloading package punkt to /tmp/llama_index...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Generating embeddings:   0%|          | 0/4149 [00:00<?, ?it/s]

**********
Trace: query
    |_query ->  3.442385 seconds
      |_retrieve ->  1.241585 seconds
      |_synthesize ->  2.200355 seconds
        |_templating ->  7.5e-05 seconds
        |_llm ->  2.189853 seconds
**********
**********
Trace: query
    |_query ->  3.481681 seconds
      |_retrieve ->  1.104158 seconds
      |_synthesize ->  2.376781 seconds
        |_templating ->  7.9e-05 seconds
        |_llm ->  2.371367 seconds
**********
**********
Trace: query
    |_query ->  2.372142 seconds
      |_retrieve ->  0.615845 seconds
      |_synthesize ->  1.755029 seconds
        |_templating ->  4.2e-05 seconds
        |_llm ->  1.750945 seconds
**********
**********
Trace: query
    |_query ->  5.302048 seconds
      |_retrieve ->  1.117908 seconds
      |_synthesize ->  4.174141 seconds
        |_templating ->  5.6e-05 seconds
        |_llm ->  4.166784 seconds
**********
**********
Trace: query
    |_query ->  10.282434 seconds
      |_retrieve ->  1.4083 seconds
      |_synthesize

Eta:2023-10-02 01:31:05.076869 |█████████████████████████████| 100.0% (5/5) [00:02<00:00,  1.68it/s]
Eta:2023-10-02 01:31:20.439972 |███████████████████████████| 100.0% (20/20) [00:15<00:00,  1.30it/s]
Eta:2023-10-02 01:31:23.811769 |█████████████████████████████| 100.0% (5/5) [00:03<00:00,  1.50it/s]
Eta:2023-10-02 01:31:36.354929 |███████████████████████████| 100.0% (20/20) [00:12<00:00,  1.60it/s]


**********
Trace: query
    |_query ->  2.187333 seconds
      |_retrieve ->  0.656751 seconds
      |_synthesize ->  1.530191 seconds
        |_templating ->  4.4e-05 seconds
        |_llm ->  1.525733 seconds
**********
**********
Trace: query
    |_query ->  3.02726 seconds
      |_retrieve ->  1.115723 seconds
      |_synthesize ->  1.910754 seconds
        |_templating ->  8.1e-05 seconds
        |_llm ->  1.90358 seconds
**********
**********
Trace: query
    |_query ->  2.159267 seconds
      |_retrieve ->  0.836572 seconds
      |_synthesize ->  1.322257 seconds
        |_templating ->  6.4e-05 seconds
        |_llm ->  1.317827 seconds
**********
**********
Trace: query
    |_query ->  4.700037 seconds
      |_retrieve ->  0.716722 seconds
      |_synthesize ->  3.982898 seconds
        |_templating ->  4.1e-05 seconds
        |_llm ->  3.978504 seconds
**********
**********
Trace: query
    |_query ->  2.374605 seconds
      |_retrieve ->  0.677691 seconds
      |_synthesize 

Eta:2023-10-02 01:33:03.774008 |█████████████████████████████| 100.0% (5/5) [00:02<00:00,  1.69it/s]
Eta:2023-10-02 01:33:22.261392 |███████████████████████████| 100.0% (30/30) [00:18<00:00,  1.62it/s]
Eta:2023-10-02 01:33:25.207443 |█████████████████████████████| 100.0% (5/5) [00:02<00:00,  1.74it/s]
Eta:2023-10-02 01:33:43.742572 |███████████████████████████| 100.0% (30/30) [00:18<00:00,  1.62it/s]


**********
Trace: query
    |_query ->  1.95206 seconds
      |_retrieve ->  0.680254 seconds
      |_synthesize ->  1.271388 seconds
        |_templating ->  6e-05 seconds
        |_llm ->  1.265284 seconds
**********
**********
Trace: query
    |_query ->  3.607624 seconds
      |_retrieve ->  0.889689 seconds
      |_synthesize ->  2.717485 seconds
        |_templating ->  6.1e-05 seconds
        |_llm ->  2.70872 seconds
**********
**********
Trace: query
    |_query ->  2.516423 seconds
      |_retrieve ->  1.102323 seconds
      |_synthesize ->  1.413623 seconds
        |_templating ->  7.9e-05 seconds
        |_llm ->  1.403792 seconds
**********
**********
Trace: query
    |_query ->  3.929991 seconds
      |_retrieve ->  0.663254 seconds
      |_synthesize ->  3.266321 seconds
        |_templating ->  6.1e-05 seconds
        |_llm ->  3.260305 seconds
**********
**********
Trace: query
    |_query ->  8.987166 seconds
      |_retrieve ->  0.684674 seconds
      |_synthesize ->

Eta:2023-10-02 01:36:12.060839 |█████████████████████████████| 100.0% (5/5) [00:03<00:00,  1.48it/s]
Eta:2023-10-02 01:36:43.282921 |███████████████████████████| 100.0% (50/50) [00:31<00:00,  1.60it/s]
Eta:2023-10-02 01:36:46.788812 |█████████████████████████████| 100.0% (5/5) [00:03<00:00,  1.45it/s]
Eta:2023-10-02 01:37:16.174384 |███████████████████████████| 100.0% (50/50) [00:29<00:00,  1.70it/s]


In [ ]:
all_data_filepath = os.path.join(save_dir, f"{web_title}_all_data.pkl")
with open(all_data_filepath, "wb") as f:
    pickle.dump(all_data, f)

# The retrievals with 0 relevant context really can't be optimized, removing gives a diff view
plot_graphs(
    all_data=all_data,
    save_dir=os.path.join(save_dir, "results_zero_removed"),
    show=False,
    remove_zero=True,
)
plot_graphs(
    all_data=all_data,
    save_dir=os.path.join(save_dir, "results_zero_not_removed"),
    show=False,
    remove_zero=False,
)

## Example Results Q&A Evals (actual results in experiment_data)

The **Q&A Eval** runs at the highest level of did you get the question answer correct  based on the data:

<center>
    <p style="text-align:center">
        <img alt="phoenix data" src="https://storage.googleapis.com/arize-assets/phoenix/assets/images/percentage_incorrect_plot.png" />
    </p>
</center>



## Example Results Retrieval Eval  (actual results in experiment_data)

**Retrieval Eval:** Precision @ K The retrieval analysis example is below, iterates through the chunk sizes, K (4/6/10), retrieval method
The eval checks whether the retrieved chunk is relevant and has a chance to answer the question

<center>
    <p style="text-align:center">
        <img alt="phoenix data" src="https://storage.googleapis.com/arize-assets/phoenix/assets/images/all_mean_precisions.png" />
    </p>
</center>

## Example Results Latency  (actual results in experiment_data)

The latency can highly varied based on retrieval approaches, below are latency maps

<center>
    <p style="text-align:center">
        <img alt="phoenix data" src="https://storage.googleapis.com/arize-assets/phoenix/assets/images/median_latency_all.png" />
    </p>
</center>
